In [58]:
from dotenv import load_dotenv
load_dotenv()

True

In [59]:
import os
import sqlite3
from langchain import hub
from langchain.sql_database import SQLDatabase
from langchain_core.messages import  HumanMessage
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import TextLoader
from langchain.chains import create_extraction_chain
from langchain_community.agent_toolkits import create_sql_agent

In [60]:
loader = TextLoader("./data.txt")
docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs) #if we need to split doc.
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

schema = {
    "properties": {
        "name": {"type": "string"},
        "phone_number": {"type": "integer"},
    },
    "required": [],
}

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [61]:
contextualize_q_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. """
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [62]:
# DB_URI = "sqlite:///./Chinook.db"
# print(DB_URI)
# db = SQLDatabase.from_uri(DB_URI)

# create_data_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# create_agent_executor = create_sql_agent(create_data_llm, db=db, agent_type="openai-tools", verbose=False)
# create_agent_executor.invoke(
#     f"""Create Table name appointment table if already not exist. where colums are Phone Number : Integer, Name : String, Date : Date , Time:time fields."""
# )

In [63]:
qa_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. \
    {context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever
    )
    | qa_prompt
    | llm
)

In [64]:
chat_history = []

llmdata = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def check_phone_name(converted_answer):
  if converted_answer.get('phone_number') and converted_answer.get('phone_number') != 1234567890 and converted_answer.get('phone_number') != 0 and len(converted_answer.get('phone_number'), 0) > 9:
    print("Adding Phone and name into database")
    # DB_URI = f"mysql://{os.environ['DB_USER']}:{os.environ['DB_PASSWORD']}@{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_NAME']}"
    # base_dir = os.getcwd()
    
    # DB_URI = f"sqlite:///{os.path.join(base_dir, 'Chinook.db')}"
    
    DB_URI = "/Users/himalaypatel/Documents/HimalayWork/Learning/LangChain-AI-Voice-Call-asistant/Chinook.db"
    
    print(DB_URI)
    connection = sqlite3.connect(DB_URI)
    
    cursor = connection.cursor()
    
    appointment_data = {
    'phone_number': converted_answer.get('phone_number'),
    'name': converted_answer.get('name'),
    }
    
    insert_query = '''
    INSERT INTO appointment (phone_number, name)
    VALUES (?, ?)
    '''
    cursor.execute(insert_query, (
    appointment_data['phone_number'],
    appointment_data['name'],
    ))
    
    connection.commit()
    connection.close()
    
    #------------inserting data into database is not working as expected. so adding them manually
    # db = SQLDatabase.from_uri(DB_URI)
  
    # insert_data_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    # agent_executor = create_sql_agent(insert_data_llm, db=db, agent_type="openai-tools", verbose=True)
    # agent_executor.invoke(
    #   # f"Add phone_number : {converted_answer.get('phone_number')} and name : {converted_answer.get('name')} insert data into appointment table make date and time as current date and time."
    #   f"""Insert the following data into the appointment table:

    #   - Phone Number: {converted_answer.get('phone_number')}
    #   - Name: {converted_answer.get('name')}

    #   """
    # )
    
for i in range(5):
  
  question = input("Himalay: ")
  print("Himlay: ", question)
  
  
  chain = create_extraction_chain(schema, llmdata)
  converted_answer = chain.run(question)
  print(f"user converted answer - {converted_answer}")
  if len(converted_answer) > 0:
    check_phone_name(converted_answer[0])
  
  ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
  
  
  print("AI Response: ",ai_msg)
  print("--------------------------------------------------------------")
  chat_history.extend([HumanMessage(content=question), ai_msg])

Himlay:  Hello
user converted answer - [{'name': 'John Doe'}]
AI Response:  content='Hi, this is South Bay Dental Office, how may I help you?'
--------------------------------------------------------------
Himlay:  i want to book an apointment
user converted answer - []
AI Response:  content='Step 1: Are you a new or existing patient?'
--------------------------------------------------------------
Himlay:  new
user converted answer - [{'name': 'John Doe', 'phone_number': 1234567890}]
AI Response:  content='Step 2: Thank you for choosing South Bay Dental Office. May I have your name and phone number, please?'
--------------------------------------------------------------
Himlay:  aditya 8974509845
user converted answer - [{'name': 'aditya', 'phone_number': 8974509845}]
Adding Phone and name into database
/Users/himalaypatel/Documents/HimalayWork/Learning/LangChain-AI-Voice-Call-asistant/Chinook.db
AI Response:  content="Thank you, Aditya, for providing your phone number. Now, let's move